In [4]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [5]:
#since each feature (value) in a sample has a different range, take the mean of each column
#and subtract it from each value in the column, then divide the standard deviation

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

#quantities used for normalizing the test data are computed using the training data!
test_data -= mean
test_data -= std

In [6]:
# the less data available the worse the overfitting will be, so a small network is used to mitigate the potential problem

from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) #the neural network ends with a single unit w/ no activation. 
    #typical setup for scalar regression (when trying to predict a single continuous value) like house price in this case
    #the network is not confined to output a value within a range (in the case of a sigmoid function)
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) #mse is a popular loss function for regression
    return model

In [ ]:
#since the training data is small, just splitting the data into training and validation will create a large variance in validations scores
#to combat this, k-fold cross-validation will be used

import numpy as np

k = 4
num_val_samples = len(train_data) // k #floor division
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples], 
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
    all_scores.append(val_mae)

processing fold # 0


In [ ]:
print(len(all_scores))